In [ ]:
!git clone https://github.com/YunHaaaa/koelectra_hatespeech.git KoELECTRA_new

In [ ]:
%cd KoELECTRA_new/finetune

In [ ]:
!pip install --upgrade transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import argparse
import json
import logging
import os
import glob
import re

import numpy as np
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from fastprogress.fastprogress import master_bar, progress_bar

from transformers import (
    AdamW,
    get_linear_schedule_with_warmup
)

from src import (
    CONFIG_CLASSES,
    TOKENIZER_CLASSES,
    MODEL_FOR_SEQUENCE_CLASSIFICATION,
    init_logger,
    set_seed,
    compute_metrics
)
from processor import seq_cls_load_and_cache_examples as load_and_cache_examples
from processor import seq_cls_tasks_num_labels as tasks_num_labels
from processor import seq_cls_processors as processors
from processor import seq_cls_output_modes as output_modes
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score



logger = logging.getLogger(__name__)


In [ ]:
# 데이터셋 분리

# import pandas as pd
# from sklearn.model_selection import train_test_split


# # Load the dataset
# data = pd.read_csv('./df.csv').dropna().reset_index(drop=True)
# data["content"] = data["content"].astype('string')

# # Split into train, test, and dev sets
# x_train, x_test, y_train, y_test = train_test_split(data["content"], data["label"], test_size=0.15)
# x_train, x_dev, y_train, y_dev = train_test_split(x_train, y_train, test_size=0.2)

# # Create DataFrames for train, val, and dev sets
# train_data = pd.DataFrame({"content": x_train, "label": y_train})
# val_data = pd.DataFrame({"content": x_dev, "label": y_dev})
# dev_data = pd.DataFrame({"content": x_test, "label": y_test})

# # Save train, val, and dev sets as TSV files
# train_data.to_csv('train.tsv', sep='\t', index=False)
# val_data.to_csv('val.tsv', sep='\t', index=False)
# dev_data.to_csv('dev.tsv', sep='\t', index=False)


In [ ]:
class Hyperparameters:
    def __init__(self):
        # 작업(task) 이름
        self.task = "hate-speech"
        # 데이터 경로
        self.data_dir = "data"
        # 체크포인트(모델 가중치) 저장 경로
        self.ckpt_dir = "ckpt"
        # 학습용 파일명
        self.train_file = "train.tsv"
        # 검증용 파일명
        self.dev_file = "" # "dev.csv"
        # 테스트용 파일명
        self.test_file = "val.tsv"
        # 학습 중 테스트 성능 평가 여부
        self.evaluate_test_during_training = True
        # 모든 체크포인트 평가 여부
        self.eval_all_checkpoints = True
        # 옵티마이저 저장 여부
        self.save_optimizer = False
        # 소문자 변환 여부
        self.do_lower_case = False
        # 학습 수행 여부
        self.do_train = True
        # 평가 수행 여부
        self.do_eval = True
        # 최대 시퀀스 길이
        self.max_seq_len = 128
        # 학습 에폭 수
        self.num_train_epochs = 10
        # 가중치 감소 비율
        self.weight_decay = 0.0
        # 그래디언트 누적 스텝 수
        self.gradient_accumulation_steps = 1
        # Adam 옵티마이저의 epsilon 값
        self.adam_epsilon = 1e-8
        # 학습 시작 전 워밍업 비율
        self.warmup_proportion = 0
        # 최대 학습 스텝 수
        self.max_steps = -1
        # 그래디언트 클리핑을 위한 최대 노름 값
        self.max_grad_norm = 1.0
        # CUDA 사용 여부
        self.no_cuda = False
        # 모델 타입
        self.model_type = "koelectra-base-v3"
        # 모델 경로 또는 이름
        self.model_name_or_path = "monologg/koelectra-base-v3-discriminator"
        # 출력 디렉토리 경로
        self.output_dir = "koelectra-base-v3-hate-speech-ckpt"
        # 시드 값
        self.seed = 42
        # 학습 배치 크기
        self.train_batch_size = 32
        # 평가 배치 크기
        self.eval_batch_size = 64
        # 로깅 간격 (몇 스텝마다 로그를 출력할지)
        self.logging_steps = 100
        # 체크포인트 저장 간격 (몇 스텝마다 체크포인트를 저장할지)
        self.save_steps = 100
        # 학습률
        self.learning_rate = 5e-5
        # 사용할 디바이스
        self.device = ""


args = Hyperparameters()
print(args.task)  # 출력: "hate-speech"


hate-speech


In [ ]:
# GPU or CPU device 설정
args.device = "cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu"

In [ ]:
args.device

'cuda'

In [ ]:
def evaluate(args, model, eval_dataset, mode, global_step=None):
    # 결과를 저장할 딕셔너리
    results = {}
    # 정확도 초기화
    accuracy = 0.0
    # 순차적으로 데이터를 샘플링하는 Sampler 생성
    eval_sampler = SequentialSampler(eval_dataset)
    # DataLoader 생성
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

    # Eval!
    if global_step != None:
        logger.info("***** Running evaluation on {} dataset ({} step) *****".format(mode, global_step))
    else:
        logger.info("***** Running evaluation on {} dataset *****".format(mode))
    logger.info("  Num examples = {}".format(len(eval_dataset)))
    logger.info("  Eval Batch size = {}".format(args.eval_batch_size))
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    out_label_ids = None

    for batch in progress_bar(eval_dataloader):
        # 평가
        model.eval()
        # 배치의 텐서를 GPU로 이동
        batch = tuple(t.to(args.device) for t in batch)

        with torch.no_grad():
            # 모델 입력값 설정
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "labels": batch[3]
            }
            # Distilkobert, XLM-Roberta는 segment_ids를 사용하지 않음
            if args.model_type not in ["distilkobert", "xlm-roberta"]:
                inputs["token_type_ids"] = batch[2]  
            # 모델에 입력 데이터 전달하여 결과 얻기
            outputs = model(**inputs)
            # 평가 손실과 로짓(모델의 출력) 추출
            tmp_eval_loss, logits = outputs[:2]

            # 평가 손실 누적
            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        # 첫 번째 배치의 로짓 값, 실제 레이블 값 저장
        if preds is None:
            preds = logits.detach().cpu().numpy()
            out_label_ids = inputs["labels"].detach().cpu().numpy()
        # 이후 배치의 로짓 값, 실제 레이블 값 추가
        else:
            preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(out_label_ids, inputs["labels"].detach().cpu().numpy(), axis=0)
    
    # 평균 평가 손실 계산
    eval_loss = eval_loss / nb_eval_steps
    # 분류 작업의 경우, 가장 높은 로짓 값을 가진 클래스 선택
    if output_modes[args.task] == "classification":
        preds = np.argmax(preds, axis=1)
    # 회귀 작업의 경우, 로짓 값을 스칼라로 변환
    elif output_modes[args.task] == "regression":
        preds = np.squeeze(preds)
    
    # 메트릭 계산
    result = compute_metrics(args.task, out_label_ids, preds)
    # 정확도 계산
    accuracy = accuracy_score(out_label_ids, preds)
    # 정밀도 계산
    precision = precision_score(out_label_ids, preds)
    # 재현율 계산
    recall = recall_score(out_label_ids, preds)
    # ROC-AUC 계산
    auc_roc = roc_auc_score(out_label_ids, preds)

    # 결과 딕셔너리에 메트릭 결과 업데이트
    results.update(result)
    results.update({"accuracy": accuracy, "precision": precision, "recall": recall, "auc_roc": auc_roc})

    # 결과를 저장할 디렉토리 경로 생성
    output_dir = os.path.join(args.output_dir, mode)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 결과 로그 , 파일에 기록
    output_eval_file = os.path.join(output_dir, "{}-{}.txt".format(mode, global_step) if global_step else "{}.txt".format(mode))
    with open(output_eval_file, "w") as f_w:
        logger.info("***** Eval results on {} dataset *****".format(mode))
        for key in sorted(results.keys()):
            logger.info("  {} = {}".format(key, str(results[key])))
            f_w.write("  {} = {}\n".format(key, str(results[key])))

    return results


In [ ]:
def train(args,
          model,
          train_dataset,
          dev_dataset=None,
          test_dataset=None):
    # 무작위 샘플링을 위한 Sampler를 생성
    train_sampler = RandomSampler(train_dataset)
    # DataLoader를 생성하여 데이터를 미니배치로 나눔
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size)
    if args.max_steps > 0:
        t_total = args.max_steps
        # 전체 스텝의 수를 계산
        # 스텝 수는 미니배치의 수와 그래디언트 누적 스텝 수, 에폭 수에 따라 결정
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    # 바이어스(bias)와 레이어 정규화 층의 가중치(LayerNorm.weight)에 대해서 가중치 감소를 적용하지 않음
    no_decay = ['bias', 'LayerNorm.weight']
    # 옵티마이저에게 전달될 파라미터와 해당 파라미터에 적용할 가중치 감소를 설정
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
         'weight_decay': args.weight_decay},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    # AdamW 옵티마이저를 생성
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    # warmup 및 decay가 적용된 선형 스케줄러를 생성
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(t_total * args.warmup_proportion), num_training_steps=t_total)

    # 저장된 옵티마이저와 스케줄러 상태가 존재하는 경우 해당 상태를 불러와서 모델을 이어서 학습
    if os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt")) and os.path.isfile(
            os.path.join(args.model_name_or_path, "scheduler.pt")
    ):
        # 저장된 옵티마이저와 스케줄러 상태를 불러옴
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Total train batch size = %d", args.train_batch_size)
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)
    logger.info("  Logging steps = %d", args.logging_steps)
    logger.info("  Save steps = %d", args.save_steps)

    global_step = 0
    tr_loss = 0.0

    # 모델의 그래디언트를 초기화
    model.zero_grad()
    mb = master_bar(range(int(args.num_train_epochs)))
    # 각 훈련 에포크에 대한 반복
    for epoch in mb:
        epoch_iterator = progress_bar(train_dataloader, parent=mb)
        for step, batch in enumerate(epoch_iterator):
            # 모델을 훈련
            model.train()
            # 배치의 텐서들을 GPU로 이동
            batch = tuple(t.to(args.device) for t in batch)
            # 모델 인풋 생성
            inputs = {
                # 입력 토큰의 ID
                "input_ids": batch[0],
                # 어텐션 마스크
                "attention_mask": batch[1],
                # 레이블
                "labels": batch[3]
            }
            # Distilkobert, XLM-Roberta는 segment_ids를 사용하지 않음
            if args.model_type not in ["distilkobert", "xlm-roberta"]:
                inputs["token_type_ids"] = batch[2]  
            # 모델의 순전파를 수행
            outputs = model(**inputs)

            # 손실 값
            loss = outputs[0]

            # 그래디언트 누적을 위해 손실 값을 나눔
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            # 역전파를 수행하여 그래디언트를 계산
            loss.backward()
            # 훈련 손실 값을 누적
            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0 or (
                    len(train_dataloader) <= args.gradient_accumulation_steps
                    and (step + 1) == len(train_dataloader)
            ):  
                # 그래디언트 클리핑을 수행
                torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)

                # 옵티마이저의 파라미터 업데이트
                optimizer.step()
                # 스케줄러 업데이트
                scheduler.step()
                # 그래디언트 초기화
                model.zero_grad()
                # 스텝 수 증가
                global_step += 1

                if args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    if args.evaluate_test_during_training:
                        # 테스트 데이터셋에 대한 성능 평가를 수행
                        evaluate(args, model, test_dataset, "test", global_step)
                    else:
                        # 개발 데이터셋에 대한 성능 평가를 수행
                        evaluate(args, model, dev_dataset, "dev", global_step)

                if args.save_steps > 0 and global_step % args.save_steps == 0:
                    # 모델 체크포인트 저장
                    output_dir = os.path.join(args.output_dir, "checkpoint-{}".format(global_step))
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )
                    # 모델의 구성 저장
                    model_to_save.save_pretrained(output_dir)
                    # 학습 인자 저장
                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to {}".format(output_dir))

                    # 옵티마이저와 스케줄러 상태 저장
                    if args.save_optimizer:
                        torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                        torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                        logger.info("Saving optimizer and scheduler states to {}".format(output_dir))

            # 지정된 최대 스텝 수에 도달했을 때 훈련을 조기 종료
            if args.max_steps > 0 and global_step > args.max_steps:
                break

        mb.write("Epoch {} done".format(epoch + 1))

        if args.max_steps > 0 and global_step > args.max_steps:
            break
    # 전역 스텝 수, 훈련 손실의 평균 반환
    return global_step, tr_loss / global_step



In [ ]:
def main():
    logger.info("Training/evaluation parameters {}".format(args))

    args.output_dir = os.path.join(args.ckpt_dir, args.output_dir)
    # 로그 초기화
    init_logger()
    # 시드 설정
    set_seed(args)

    # 전처리기 생성
    processor = processors[args.task](args)
    # 레이블 가져오기
    labels = processor.get_labels()
    # 회귀 문제인 경우, 설정된 레이블 수(num_labels)를 사용하여 설정 객체 생성
    if output_modes[args.task] == "regression":
        config = CONFIG_CLASSES[args.model_type].from_pretrained(
            args.model_name_or_path,
            num_labels=tasks_num_labels[args.task]
        )
    # 분류 문제인 경우, 설정된 레이블 수(num_labels) 및 레이블-인덱스 매핑 정보(id2label, label2id)를 사용하여 설정 객체 생성
    else:
        config = CONFIG_CLASSES[args.model_type].from_pretrained(
            args.model_name_or_path,
            num_labels=tasks_num_labels[args.task],
            id2label={str(i): label for i, label in enumerate(labels)},
            label2id={label: i for i, label in enumerate(labels)},
        )
    # 토크나이저 불러오기
    tokenizer = TOKENIZER_CLASSES[args.model_type].from_pretrained(
        args.model_name_or_path,
        do_lower_case=args.do_lower_case
    )
    # pretrained 모델 불러오기
    model = MODEL_FOR_SEQUENCE_CLASSIFICATION[args.model_type].from_pretrained(
        args.model_name_or_path,
        config=config
    )

    # GPU or CPU
    args.device = "cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu"
    model.to(args.device)

    # 데이터셋 로드
    train_dataset = load_and_cache_examples(args, tokenizer, mode="train") if args.train_file else None
    dev_dataset = load_and_cache_examples(args, tokenizer, mode="dev") if args.dev_file else None
    test_dataset = load_and_cache_examples(args, tokenizer, mode="test") if args.test_file else None

    # 검증 데이터셋이 없는 경우, 테스트셋만 사용하도록 설정
    if dev_dataset == None:
        args.evaluate_test_during_training = True  # If there is no dev dataset, only use testset

    # 훈련 실행 여부 확인 후 훈련 함수 호출
    if args.do_train:
        global_step, tr_loss = train(args, model, train_dataset, dev_dataset, test_dataset)
        logger.info(" global_step = {}, average loss = {}".format(global_step, tr_loss))

    results = {}
    # 평가 실행 여부 확인 후 평가 함수 호출
    if args.do_eval:
        # 체크포인트 파일들의 경로를 나타내는 리스트
        checkpoints = list(os.path.dirname(c) for c in
                           sorted(glob.glob(args.output_dir + "/**/" + "pytorch_model.bin", recursive=True),
                                  key=lambda path_with_step: list(map(int, re.findall(r"\d+", path_with_step)))[-1]))
        # eval_all_checkpoints가 False인 경우, 가장 최근 체크포인트 하나만 사용
        if not args.eval_all_checkpoints:
            checkpoints = checkpoints[-1:]
        else:
            logging.getLogger("transformers.configuration_utils").setLevel(logging.WARN)  # Reduce logging
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        
        # 체크포인트마다 작업 수행
        for checkpoint in checkpoints:
            # 해당 체크포인트 모델 생성
            global_step = checkpoint.split("-")[-1]
            model = MODEL_FOR_SEQUENCE_CLASSIFICATION[args.model_type].from_pretrained(checkpoint)
            model.to(args.device)
            # 테스트 데이터셋을 평가
            result = evaluate(args, model, test_dataset, mode="test", global_step=global_step)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)
        # 평가 결과를 파일에 저장
        output_eval_file = os.path.join(args.output_dir, "eval_results.txt")
        with open(output_eval_file, "w") as f_w:
            if len(checkpoints) > 1:
                for key in sorted(results.keys(), key=lambda key_with_step: (
                        "".join(re.findall(r'[^_]+_', key_with_step)),
                        int(re.findall(r"_\d+", key_with_step)[-1][1:])
                )):
                    f_w.write("{} = {}\n".format(key, str(results[key])))
            else:
                for key in sorted(results.keys()):
                    f_w.write("{} = {}\n".format(key, str(results[key])))



In [ ]:
if __name__ == '__main__':
    main()

/content/drive/MyDrive/KoELECTRA_new/finetune


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-15-5bec3018ea43>", line 3, in <cell line: 2>
    main()
  File "<ipython-input-14-3d8bc88cecf8>", line 45, in main
    global_step, tr_loss = train(args, model, train_dataset, dev_dataset, test_dataset)
  File "<ipython-input-13-ac157b9b3673>", line 80, in train
    evaluate(args, model, test_dataset, "test", global_step)
  File "<ipython-input-12-fc406af157a4>", line 60, in evaluate
    os.makedirs(output_dir)
  File "/usr/lib/python3.10/os.py", line 215, in makedirs
    makedirs(head, exist_ok=exist_ok)
  File "/usr/lib/python3.10/os.py", line 215, in makedirs
    makedirs(head, exist_ok=exist_ok)
  File "/usr/lib/python3.10/os.py", line 225, in makedirs
    mkdir(name, mode)
OSError: [Errno 107] Transport endpoint is not connected: 'ckpt'

During handling of the above 

In [ ]:
# 토크나이저 로드
tokenizer = TOKENIZER_CLASSES[args.model_type].from_pretrained(
        args.model_name_or_path,
        do_lower_case=args.do_lower_case
    )

In [ ]:
# f1 score가 가장 높은 파일 찾기
import os

directory = '/content/drive/MyDrive/KoELECTRA_new/finetune/ckpt/koelectra-base-v3-hate-speech-ckpt/test'
max_f1_score = -1
best_file_name = ''

for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r') as file:
            contents = file.read()
            f1_score = float(contents.split('f1 = ')[1].split('\n')[0])
            if f1_score > max_f1_score:
                max_f1_score = f1_score
                best_file_name = filename

print("File with the highest f1 score:", best_file_name)


In [ ]:
# 해당 체크포인트 폴더 경로 설정
global_step = best_file_name.split('-')[1].split('.')[0]
output_dir = "/content/drive/MyDrive/KoELECTRA_new/finetune/ckpt/koelectra-base-v3-hate-speech-ckpt"
ckpt_path = os.path.join(output_dir, f"checkpoint-{global_step}" )

In [ ]:
# 토크나이저 정보 저장
tokenizer.save_pretrained(ckpt_path)

In [ ]:
!pip install transformers
!huggingface-cli login

In [ ]:
!huggingface-cli repo create koELECTRA_hatespeech_v4


In [ ]:
# 모델 허깅페이스에 포팅
from huggingface_hub import HfApi
api = HfApi()

# Upload all the content from the local folder to your remote Space.
# By default, files are uploaded at the root of the repo
api.upload_folder(
    folder_path=ckpt_path,
    repo_id="Haaaaeun/koELECTRA_hatespeech_v4",
    repo_type="model",
)